In [ ]:
%pip install transformers torch

In [ ]:
%pip install transformers peft datasets bitsandbytes accelerate ipywidgets


In [ ]:
# 모델을 직접 로드하여 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("kakaocorp/kanana-nano-2.1b-instruct")
model = AutoModelForCausalLM.from_pretrained("kakaocorp/kanana-nano-2.1b-instruct")

# 텍스트를 토크나이즈하여 모델 입력 형태로 변환
input_text = "Who are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# 모델을 사용하여 출력 생성
outputs = model.generate(inputs["input_ids"], max_length=100, do_sample=True)

# 출력 디코딩
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)  # 생성된 텍스트 출력


In [ ]:
# 출력 개수 제한 + linear 계층만 필터링
for name, module in model.named_modules():
    if "linear" in str(type(module)).lower():
        print(name)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

# 1. 모델 지정
model_id = "kakaocorp/kanana-nano-2.1b-instruct"  

# 2. 토크나이저 & 모델 불러오기 (8bit 로딩 + GPU 자동 할당)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto"
)

# 3. LoRA 학습을 위한 모델 준비
model = prepare_model_for_kbit_training(model)

# 4. Kanana-nano-2.1b-instruct 전용 target_modules로 LoRA 설정

llora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=16,  # LoRA 계수
    target_modules=[
        "self_attn.q_proj", 
        "self_attn.k_proj", 
        "self_attn.v_proj", 
        "self_attn.o_proj", 
        "mlp.gate_proj", 
        "mlp.up_proj", 
        "mlp.down_proj"
    ],  # LoRA 적용할 계층들
    lora_dropout=0.05,  # 드롭아웃 비율
    bias="none",  # 편향을 어떻게 처리할지
    task_type=TaskType.CAUSAL_LM  # 언어 모델링 작업
)

# LoRA 적용된 모델 생성
model = get_peft_model(model, llora_config)

# 학습 가능한 파라미터 출력
model.print_trainable_parameters()

In [ ]:
from datasets import load_dataset

# 1. CSV 로딩
dataset = load_dataset("csv", data_files="/app/workspace/Kanana_VERSION2/midjourney2.csv", split="train")

# 2. 전처리 함수 정의
def format_prompt(example):
    prompt = f"### 질문: {example['long_prompt']}\n### 답변:"
    return tokenizer(prompt, text_target=example['short_prompt'], truncation=True, padding="max_length", max_length=512)

# 3. 전처리 적용
tokenized_dataset = dataset.map(format_prompt)


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 1. 학습 설정
training_args = TrainingArguments(
    output_dir="/app/workspace/qlora_outputs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=1,
    bf16=True,
    save_strategy="no",
    report_to="none"
)

# 2. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# 3. 학습 시작
trainer.train()


In [ ]:
model.save_pretrained("/home/piai/workspace/Kanana_VERSION2")
tokenizer.save_pretrained("/home/piai/workspace/Kanana_VERSION2")


In [1]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 1. 8bit 로딩 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=True  # 핵심!
)

# 2. Base 모델 로딩
base_model = AutoModelForCausalLM.from_pretrained(
    "kakaocorp/kanana-nano-2.1b-instruct",
    device_map="auto",
    quantization_config=bnb_config
)

# 3. LoRA adapter 붙이기
model = PeftModel.from_pretrained(base_model, "/home/piai/workspace/Kanana_VERSION2")
model.eval()

# 4. Tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("/home/piai/workspace/Kanana_VERSION2", use_fast=False)

# 5. 추론 함수
def rewrite_prompt(input_ko):
    prompt = f"### 질문: {input_ko}\n### 답변:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # 불필요한 'token_type_ids' 제거
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')

    # 출력 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,  # 생성될 최대 토큰 수
        max_length = 500,
        do_sample=False,  # 샘플링 활성화
        # temperature=0.5,  # 창의적 출력을 유도
        # top_p=0.9,  # nucleus sampling
        # top_k=50,  # 샘플링 범위 제한
        repetition_penalty=1.5,  # 반복 방지
        eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        pad_token_id=tokenizer.pad_token_id,  # 패딩 토큰 설정
        num_return_sequences=1,  # 반환할 시퀀스 개수
        no_repeat_ngram_size=2  # 2-그램 반복 방지
    )
    
    # 결과 디코딩
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return output_text


In [2]:
# 5. 테스트!
example = "창문 너머를 바라보는 고양이를 그리고싶은데, 미드저니 고도화 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


💬 결과:
 ### 질문: 창문 너머를 바라보는 고양이를 그리고싶은데, 미드저니 고도화 프롬프트 작성해줘
### 답변: 
**제목:** "창밖을 내다보며 생각에 잠긴 매력적인 검정색 코비 푸들"
**
### 설명:
이 그림에서는 귀여운 성격의 소유자이자 사랑스러운 반려동물인 푸들이 등장합니다. 그의 털 색깔과 외모가 독특한 개성으로 돋보이며, 특히 눈길을 끄는 것은 그 모습입니다.
**
#### 설정 및 배경 정보  
- **장소**: 집 안에서 가족들과 함께 지내고 있는 가정집 실내 환경,
  - 따뜻하고 아늑하며 편안함을 주는 분위기 연출,
    ```markdown
```json
{
   "$model": null,

   // 기본 옵션 값 초기설정

}
```
---
## Midjourney Prompt for Generating the Image with Advanced Prompts:

1. Create a detailed illustration of an adorable black and white Poodle dog sitting in front of his home window.

2. The image should depict him gazing out into nature or looking at family members outside from inside their house while being gently petted by them on couches if available as part of your prompt to enhance realism further using advanced features like photorealism settings etc., which can be set through additional parameters when generating images via midjourneys API v5 later versions after version V4 release date mentioned belo

In [7]:
# 5. 테스트!
example = "현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

Both `max_new_tokens` (=300) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


💬 결과:
 ### 질문: 현대식 화려한 건물을 그리고 싶은데 미드저니 프롬프트 작성해줘
### 답변: Create a detailed digital illustration of an elegant modern building. The structure should be designed in the style of contemporary architecture, with sleek lines and precise proportions that reflect its sophisticated design language.
#### Description:
The main focus is on creating intricate details to enhance visual interest while maintaining clarity throughout different elements within your artwork.

1. **Building Design**:

   - Use bold colors such as metallic shades like silver or gold for walls,
     polished metal accents,

2.  Optimize lighting effects by using bright light sources from various angles

3.    Incorporate natural features through greenery plants at appropriate locations inside buildings 
4.      Add architectural detailing including columns , arches 

5. Constructing complex shapes around windows which are large enough so they provide ample space without being too overwhelming  
6. Enhance depth perception

In [6]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람, 미드저니 프롬프트 작성해줘
### 답변: "A detailed digital illustration of a person in an elegant library attire seated at the foot of an oak table. The warm glow from their piano illuminates books and shelves against dark wooden walls adorned with bookshelves."
#### 추가 설명:
- **도서와 음악의 조화**: 책과 악기가 공존하는 장면을 그려달라는 요청.
- *피아니스트*는 고전적인 복장을 하고 있으며, 분위기 잡힌 서재 한가운데에 자리잡고 있다.
  - 밝은 빛이 들어오는 창문 근처라 따뜻한 느낌이다.

미드 저니를 통해 이 이미지를 생성하면 다음과 같은 결과물을 얻을 수 있을 것입니다:

!


In [7]:
# 5. 테스트!
example = "풀숲에서 똥 싸다가 눈마주쳐서 놀란 고양이 그려줘줘줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 풀숲에서 똥 싸다가 눈마주쳐서 놀란 고양이 그려줘줘줘, 미드저니 프롬프트 작성해줘
### 답변: "A whimsical scene featuring a curious cat in the forest. The central subject is an adorable gray and white feline with large eyes gazing directly at viewers while sitting on grass near lush green foliage under starry night sky."
#### Midjourney Prisma Prompt:
"A charming close-up of your favorite furry friend up to their chins inside dense trees & vibrant leaves against dark blue background creating mystery w/ stars lighting it all." 
Feel free to adjust for specific details! 🐱🌳😊

 ### 추가 설명 및 예시 이미지 링크:

**추가 설명:** 이 프라이머리는 귀여운 시각적 요소를 강조하여 중점적으로 보여줄 수 있도록 구성했습니다.
- **고양이를


In [8]:
# 5. 테스트!
example = "도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 도서관에서 피아노 치고 있는 사람을 그리고 싶은데데데 미드저니 프롬프트 작성해줘, 미드저니 프롬프트 작성해줘
### 답변: "A serene scene of a person lost in music within the tranquil atmosphere of an ancient library. The subject is centered and surrounded by wooden columns with bookshelves at their feet while soft light illuminates everything from above."
#### 추가 설명:
- **도입부**: 사람이 책과 음악 사이에 평화롭게 존재하는 장면.
- *중심 요소*: 중심이 되는 인물(피아니스), 그의 자세와 주변 환경인 고요한 도서실 내 분위기 강조.
  - 조명 방식으로 따뜻하고 차분함 표현 (천장의 창이나 전등).
  
---

**미드져너 이미지 예시:** 
1) 한 남자가 검정색


In [9]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습, 미드저니 프롬프트 작성해줘
### 답변: "A whimsical scene of five playful kittens skydiving together. Each kitten is a miniature world with their own unique color palette and accessories against an ethereal blue backdrop filled with stars."
#### Midjourney Prisma Prompt:
1. A cute cartoon image featuring three white kittens in gray suits jumping off from high above.
2. The cats are surrounded by bubbles adorned with colorful balloons on the ground below them while birds fly around to add movement.

This prompt captures the joyous spirit behind each kitty's action as they leap into mid-air during your journey! 🐱🚀✨️

### 추가 설명 및 예시 이미지 링크:

- **미적 요소**: Colorful balloon decorations enhance playfulness; warm background colors like


In [10]:
# 5. 테스트!
example = "스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘줘줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 스카이 다이빙하고 있는 아기 고양이 5마리가 하늘에서 별을 만든 모습을 그리고 싶은데 미드저니 프롬프트 작성해줘줘줘, 미드저니 프롬프트 작성해줘
### 답변: 
"An enchanting digital illustration featuring a playful group of five baby cats skydiving together. Each cat is adorned with vibrant blue and orange wings against an ethereal backdrop filled with stars or clouds under the light from two bright yellow lights hanging overhead.

The scene captures their joyful spirits as they soar into space in synchronized movements while maintaining close proximity to one another."
  
**미드 저니(MidJourney) 사용 시:** `a whimsical portrait depicting four adorable kittens jumping off a building towards starry skies; each kitten has colorful wing decorations & holds hands amidst falling leaves`
```json
{
    "prompt": "<an enchantering digital artwork showcasing playfully flying kids on skiliding through cloudy skys above glowing yellow lanterns


In [11]:
# 5. 테스트!
example = "지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘"
print("💬 결과:\n", rewrite_prompt(example))

💬 결과:
 ### 질문: 지프를 타고 있는 썬글라쓰 끼고, 한 손엔 위스키를 들고 있는 기린 그리고 싶은데 프롬프트 작성해줘, 미드저니 프롬프트 작성해줘
### 답변: "An elegant and sophisticated scene featuring a majestic giraffe in full view. The animal is adorned with intricate patterns against an orange backdrop while the viewer's gaze captures its intense stare through sunglasses on their headpiece.

The figure stands out from his background of blurred brown hues as he holds hands next to him holding whiskey glasses at night time under starry skies.
"
#### Midjourney Prisma Prompt:
"Monochromatic yet captivating illustration showcasing two figures standing amidst deep shadows; one clad only in black attire & another dressed for adventure (with blue jacket). They face each other directly within this dark setting but are partially obscured by tall trees reaching towards them."

This prompt should help guide your image creation process using eitherMid journe orPrisma! Let
